In [ ]:
##locally loading our fine tuned model using Sentance Transformer module
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("../finetune/fine-tuned-model-rag")

c:\Users\Jatin\Desktop\RAG-chromadb\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Importing the Chroma DB from langchain and Document for 
from langchain_chroma import Chroma
from langchain.docstore.document import Document
import pandas as pd

# Load and format data
df = pd.read_csv("../data_cleaning/quotes.csv")
from langchain.docstore.document import Document

#This is here to format the documents/the text we are going to embedd in vector db so that our fine tuned model can get a better context

docs = [
    Document(
        page_content=f'{row["quote"]} — {row["author"]} | Tags: {", ".join(eval(row["tags"]))}',
        metadata={
            "author": row["author"],
            "tags": ", ".join(eval(row["tags"]))  # 🔧 CONVERT LIST TO STRING HERE
        }
    )
    for _, row in df.iterrows()
]



# some deprevation warning were being shown because the frequent changes in langchain library
from langchain_huggingface import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(
    model_name="../finetune/fine-tuned-model-rag"
)

# Store our formated documents using our own embedding model in ChromaDB (looks like we are in full control here)
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embedding,
    persist_directory="../Chroma_db"
)

In [ ]:
# Basic testing we have not used any llm yet but we can see proficiency of our trained model just by 3-4 lines of code
retriever = vectordb.as_retriever(search_kwargs={"k":10})

results = retriever.invoke("quote about courage")

for doc in results:
    print(doc.page_content)


“we believe in ordinary acts of bravery, in the courage that drives one person to stand up for another.” — veronica roth, | Tags: inspirational-quotes, strength-and-courage
“whatever you do, you need courage. whatever course you decide upon, there is always someone to tell you that you are wrong. there are always difficulties arising that tempt you to believe your critics are right. to map out a course of action and follow it to an end requires some of the same courage that a soldier needs. peace has its victories, but it takes brave men and women to win them.” — ralph waldo emerson | Tags: courage, inspirational
“i wanted you to see what real courage is, instead of getting the idea that courage is a man with a gun in his hand. it's when you know you're licked before you begin, but you begin anyway and see it through no matter what.- atticus finch” — harper lee, | Tags: atticus-finch, courage
“life shrinks or expands in proportion to one's courage.” — anais nin | Tags: courage, life
“i

In [ ]:
from langchain.chains import RetrievalQA    #Creating our RAG chain by integrating all we have been doing
from langchain.prompts import PromptTemplate 
from langchain_groq import ChatGroq  #llm comes to the play

# Load Groq LLM
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

#Groq's decommissioned models causes a big blunder in our project but thanks to streamlit for showing the clear error
llm = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192",api_key= groq_api_key 
)


prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    You are a helpful assistant that finds and recommends quotes.
    Use only the following context (each with quote, author, and tags) to answer the question.
    
    Ignore all the irrelevant quotes think and reason deep before answering.

    Context:
    {context}
    Question: {question}
    Respond with a relevant quote, the author, and any applicable tags.
    """
)


# Build RAG chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)


In [5]:
from pprint import pprint
query = "Give me a quote about fear"
response = rag_chain.invoke(query)
pprint(response)


{'query': 'Give me a quote about fear',
 'result': "Here's a quote about fear:\n"
           '\n'
           '“nothing in life is to be feared, it is only to be understood. now '
           'is the time to understand more, so that we may fear less.” — Marie '
           'Curie | Tags: fear, life, science, understanding'}


In [ ]:
from datasets import Dataset

# Sample questions to test quote retrieval system
sample_queries = [
    "Give me a quote about being yourself",
    "Share a quote by Oscar Wilde about life or truth",
    "Quote about mistakes and love by Marilyn Monroe",
    "Tell me a funny quote about books",
    "Find a quote by Bernard M. Baruch on individuality",
    "Quote about treating inferiors by J.K. Rowling"
]

# Evaluation dataset - improved to match actual context and ground truth
eval_data = [
    {
        "question": "Give me a quote about being yourself",
        "answer": "Be yourself; everyone else is already taken.",
        "contexts": [
            '"Be yourself; everyone else is already taken." — Oscar Wilde | Tags: be-yourself, honesty, inspirational'
        ],
        "ground_truth": "Be yourself; everyone else is already taken."
    },
    {
        "question": "Share a quote by Oscar Wilde about life or truth",
        "answer": "To live is the rarest thing in the world. Most people exist, that is all.",
        "contexts": [
            '"To live is the rarest thing in the world. Most people exist, that is all." — Oscar Wilde | Tags: life'
        ],
        "ground_truth": "To live is the rarest thing in the world. Most people exist, that is all."
    },
    {
        "question": "Quote about mistakes and love by Marilyn Monroe",
        "answer": "If you can't handle me at my worst, then you sure as hell don't deserve me at my best.",
        "contexts": [
            '"I\'m selfish, impatient and a little insecure... but if you can\'t handle me at my worst..." — Marilyn Monroe | Tags: love, mistakes, life'
        ],
        "ground_truth": "I'm selfish, impatient and a little insecure. I make mistakes... but if you can't handle me at my worst, then you sure as hell don't deserve me at my best."
    },
    {
        "question": "Tell me a funny quote about books",
        "answer": "So many books, so little time.",
        "contexts": [
            '"So many books, so little time." — Frank Zappa | Tags: books, humor'
        ],
        "ground_truth": "So many books, so little time."
    },
    {
        "question": "Find a quote by Bernard M. Baruch on individuality",
        "answer": "Be who you are and say what you feel...",
        "contexts": [
            '"Be who you are and say what you feel..." — Bernard M. Baruch | Tags: individuality, be-yourself'
        ],
        "ground_truth": "Be who you are and say what you feel, because those who mind don't matter, and those who matter don't mind."
    },
    {
        "question": "Quote about treating inferiors by J.K. Rowling",
        "answer": "If you want to know what a man's like...",
        "contexts": [
            '"If you want to know what a man\'s like..." — J.K. Rowling | Tags: character'
        ],
        "ground_truth": "If you want to know what a man's like, take a good look at how he treats his inferiors, not his equals."
    }
]

#question: What the user asked.
# answer: What your RAG model responded with.
# contexts: The quote passage retrieved from your vector DB (e.g., ChromaDB).
# ground_truth: The correct answer to compare against your model's answer.

# Convert to Hugging Face Dataset
ragas_dataset = Dataset.from_list(eval_data)

# Dataset({
#     features: ['question', 'answer', 'contexts', 'ground_truth'],
#     num_rows: 6
# })



In [7]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

# LLM from Groq
llm = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192",
    api_key=groq_api_key
)

embedding = HuggingFaceEmbeddings(model_name="../finetune/fine-tuned-model-rag")

In [ ]:
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
from ragas.evaluation import evaluate

results = evaluate(
    ragas_dataset,
    metrics=[faithfulness, answer_relevancy, context_precision, context_recall],
    llm=llm,                 # LLM from Groq
    embeddings=embedding     # fine-tuned embedding model
)

print(results)


Evaluating:  88%|████████▊ | 21/24 [03:00<00:33, 11.09s/it]Exception raised in Job[12]: TimeoutError()
Exception raised in Job[20]: TimeoutError()
Evaluating: 100%|██████████| 24/24 [03:02<00:00,  7.60s/it]


{'faithfulness': 1.0000, 'answer_relevancy': 0.2967, 'context_precision': 1.0000, 'context_recall': 0.7333}


In [ ]:
pd.DataFrame(eval_data).to_csv("Evaluation_outputs.csv", index=False) # Saves the output csv to our vector db